# QBIO 401 Midterm 1
## Hirad Hosseini
## Fall 2023

<strong>1. (3 pts) It turns out that (at least for some runs) Illumina sequence files do not use all of
the possible quality scores (0 – 40). Write a function (or several functions) to determine
how many different quality scores are used in the fastq file,
“XI1_ATCACG_L001_R1_001.fastq”. In addition, output which quality scores are used
and the (non-zero) counts for these quality scores.<strong>

In [13]:
def quality_score_extractor(file_name: str) -> dict:
    with open(file_name, "r") as f_in:
        q_scores = [i for i in range(41)] #possible quality scores
        q_score_counts = [0] * 41 #counts for each of the possible quality scores
        line_count = 0
        for line in f_in:
            line_count += 1
            if line_count % 4 == 0: #quality scores are found on every fourth line
                temp = line.replace("/n", "")
                for i in range(len(temp)):
                    q_score_counts[ord(temp[i]) - 33] += 1 #increments the freq of the quality score found
        q_scores_dict = dict(zip(q_scores, q_score_counts)) #creates dict from q_scores and their freqs
        #filters dict to remove zero-count key,value pairs
        q_scores_dict = dict(filter(lambda x: x[1] != 0, q_scores_dict.items()))
        return q_scores_dict

In [14]:
scores = quality_score_extractor('XI1_ATCACG_L001_R1_001.fastq')
print(scores)

{2: 135385, 13: 118170, 18: 10000, 22: 30072, 27: 94911, 32: 195080, 38: 1041606, 40: 884776}


<strong>2. (7 pts) Does the distribution of quality scores at a site depend on the quality score at the
previous site? Write a function (or several functions) that will take as input a fastq
filename and a quality score. The function will look for all sites with the input quality
score and then record the distribution of quality scores at the next site. With the file
“XI1_ATCACG_L001_R1_001.fastq” run the function two times, first with input quality
score 2 and then with input quality score 40. Output the quality scores and their associated counts for which the associate
counts are not zero. <strong>

In [15]:
def quality_score_dist(file_name: str, q_score: int) -> dict:
    with open(file_name, "r") as f_in:
        quality_scores = [i for i in range(41)] #possible quality scores
        quality_score_counts = [0] * 41 #counts for each of the possible quality scores
        line_count = 0
        for line in f_in:
            line_count += 1
            if line_count % 4 == 0: #quality scores are found on every fourth line
                temp = line.replace("/n", "")
                for i in range(len(temp) - 1): # we need len-1 because we are looking at the next site
                    if ord(temp[i]) - 33 == q_score:
                        quality_score_counts[ord(temp[i+1]) - 33] += 1 #increments the freq of the quality score found
                        #at the next site
        q_scores_dist_dict = dict(zip(quality_scores, quality_score_counts)) #creates dict from q_scores and their freqs
        #filters dict to remove zero-count key,value pairs
        q_scores_dist_dict = dict(filter(lambda x: x[1] != 0, q_scores_dist_dict.items())) 
        return q_scores_dist_dict

In [16]:
dists_2 = quality_score_dist('XI1_ATCACG_L001_R1_001.fastq', 2)
print(dists_2)

{2: 127335, 13: 7, 18: 7953, 27: 90}


In [45]:
dists_40 = quality_score_dist('XI1_ATCACG_L001_R1_001.fastq', 40)
print(dists_40)

{2: 202, 13: 3825, 22: 243, 27: 1834, 32: 15426, 38: 223260, 40: 639986}


<strong>3. (3 pts) Write a Python function that takes as input a multiz filename. The function
computes the total number of nucleotide sites in the human sequence in the multiz file
(count indels too, sum across all blocks). Run this function on the file “fixed-multiz30-ace2.txt”.<strong>

In [3]:
import re

In [5]:
def total_nucs(file_name: str) -> int:
    total_nucs = 0
    with open(file_name, "r") as f_in:
        for line in f_in:
            if re.match("a ", line): #checks for new alignment fragment. If found, reads in human seq and finds len
                human_seq = f_in.readline()
                human_seq = re.split("\s+", human_seq) #separates line into segments based on whitespace
                human_seq = human_seq[6] #sixth segment of line always corresponds to the actual human sequence
                total_nucs += len(human_seq) #sums lens of all human sequences
        return total_nucs

In [6]:
count = total_nucs("fixed-multiz30-ace2.txt")
print(count)

59476


<strong>4. (7 pts) Write a Python function that takes as input a multiz filename and a species name.
The function computes the total number of nucleotide sites in the inputted species
which agree with the human sequence (sum across all blocks). With the file “fixedmultiz30-ace2.txt” run the function two times, first with input species “panTro5”
(chimpanzee, the closest species to human in the file) and then with species “dasNov3”
(armadillo, the furthest species from human in the file). Which species agrees more with
human? Is this what you expected?<strong>

In [7]:
def same_nucs_counter(file_name: str, species_name: str) -> int:
    same_nucs = 0
    with open(file_name, "r") as f_in:
        for line in f_in:
            if re.match("a ", line): #checks for new alignment fragment. If found, compares human & specified species
                #sequences to find number of matches
                human_seq = f_in.readline()
                human_seq = re.split("\s+", human_seq) #separates line into segments based on whitespace
                human_seq = human_seq[6] #sixth segment of line always corresponds to the actual human sequence
            if re.match(f"s {species_name}", line): #checks for aligned sequence for specified species
                species_seq = line
                species_seq = re.split("\s+", species_seq) #separates line into segments based on whitespce
                species_seq = species_seq[6] #sixth segment of line always corresponds to the actual species sequence
                for i in range(len(human_seq)): #compares human and species sequences, increments for each match
                    if human_seq[i] == species_seq[i]:
                        same_nucs += 1
        return same_nucs

In [8]:
chimp = same_nucs_counter("fixed-multiz30-ace2.txt", "panTro5")
print(chimp)

57145


In [9]:
armadillo = same_nucs_counter("fixed-multiz30-ace2.txt", "dasNov3")
print(armadillo)

30611


Based on the total number of nucleotides in each species which agree with the human sequences (summed across all blocks), it appears that the chimpanzee's sequence agrees more with humans because it has a greater number of matches. This is expected from our existing knowledge of evolution and phylogeny because chimpanzees are relatively closer ancestors to humans than armadillos, so we expect a greater similarity in their genomes.